## RNN from Scratch

In [21]:
import pandas as pd
import numpy as np

In [22]:
data = pd.read_csv('clean_weather.csv', index_col=0)
data = data.ffill()
data.head()

,tmax,tmin,rain,tmax_tomorrow
1970-01-01,60.0,35.0,0.0,52.0
1970-01-02,52.0,39.0,0.0,52.0
1970-01-03,52.0,35.0,0.0,53.0
1970-01-04,53.0,36.0,0.0,52.0
1970-01-05,52.0,35.0,0.0,50.0


In [23]:
np.random.seed(0)

i_weight = np.random.rand(1, 2)
h_weight = np.random.rand(2, 2)
o_weight = np.random.rand(2, 1)

In [24]:
temps = data['tmax'].tail(3).to_numpy()
temps

array([66., 70., 62.])

In [25]:
x0 = temps[0].reshape(1, 1)
x1 = temps[1].reshape(1, 1)
x2 = temps[2].reshape(1, 1)

In [26]:
xi_0 = x0 @ i_weight
xi_0

array([[36.22169126, 47.20249818]])

In [27]:
xh_0 = np.maximum(0, xi_0)
xh_0

array([[36.22169126, 47.20249818]])

In [28]:
xo_0 = xh_0 @ o_weight
xo_0

array([[57.94406231]])

## Time Step 1 and 2


In [29]:
xi_1 = x1 @ i_weight
xh = xh_0 @ h_weight
xh_1 = np.maximum(0, xh + xi_1)

xo_1 = xh_1 @ o_weight
xo_1

array([[124.54916092]])

In [30]:
xi_2 = x2 @ i_weight

xh = xh_1 @ h_weight

xh_2 = np.maximum(0, xh + xi_2)

xo_2 = xh_2 @ o_weight

xo_2

array([[190.94853131]])

#### Using Hyperbolic Tanh as activation function

Full Forward Pass

In [31]:
np.random.seed(0)

i_weight = np.random.rand(1, 5) / 5 - .1
h_weight = np.random.rand(5, 5) / 5 - .1
h_bias = np.random.rand(1, 5) / 5 - .1

o_weight = np.random.rand(5, 1) * 50
o_bias = np.random.rand(1, 1)

In [32]:
# Storing the values of each step
outputs = np.zeros(3)
hiddens = np.zeros((3, 5))
prev_hidden = None
sequence = data['tmax'].tail(3).to_numpy()

In [33]:
for i in range(3):
    x = sequence[i].reshape(1, 1)

    xi = x @ i_weight

    if prev_hidden is None:
        xh = xi
    else:
        xh = xi + prev_hidden @ h_weight + h_bias

    xh = np.tanh(xh)
    prev_hidden = xh
    hiddens[i, ] = xh

    xo = xh @ o_weight + o_bias
    outputs[i] = xo

/tmp/ipykernel_367500/1994984842.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  outputs[i] = xo


In [34]:
outputs

array([74.31470595, 80.66149404, 77.67852446])

In [35]:
hiddens

array([[ 0.56784618,  0.99320288,  0.87557333,  0.53166114, -0.76483255],
       [ 0.58366756,  0.99568651,  0.90034879,  0.69338529, -0.84149203],
       [ 0.5383306 ,  0.99164251,  0.86287584,  0.66091071, -0.80543591]])

#### Backward Pass | Backpropagation

In [36]:
def mse(actual, predicted):
    return np.mean((actual-predicted) ** 2)

def mse_grad(actual, predicted):
    return (predicted - actual)

In [37]:
actuals = np.array([70, 62, 65])

loss_grad = mse_grad(actuals, outputs)
loss_grad

array([ 4.31470595, 18.66149404, 12.67852446])

In [39]:
o_weight_grad, o_bias_grad, h_weight_grad, h_bias_grad, i_weight_grad = [0] * 5

In [43]:
l2_grad = loss_grad[2].reshape(1, 1)

print('\n l2_grad : ', l2_grad)
print('\n hiddend[2] : ', hiddens[2])
o_weight_grad += hiddens[2][:, np.newaxis] @ l2_grad
o_bias_grad += np.mean(l2_grad)

h2_grad = l2_grad @ o_weight.T

tanh_deriv = 1 - hiddens[2, :][np.newaxis,:] ** 2

h2_grad = np.multiply(h2_grad, tanh_deriv)

h_weight_grad += hiddens[1,:][:,np.newaxis] @ h2_grad
h_bias_grad += np.mean(h2_grad)

i_weight_grad += sequence[2].reshape(1,1).T @ h2_grad


 l2_grad :  [[12.67852446]]

 hiddend[2] :  [ 0.5383306   0.99164251  0.86287584  0.66091071 -0.80543591]


In [44]:
l1_grad = loss_grad[1].reshape(1,1)

o_weight_grad += hiddens[1][:, np.newaxis] @ l1_grad
o_bias_grad += np.mean(l1_grad)

h1_grad = l1_grad @ o_weight.T

h1_grad += h2_grad @ h_weight.T

tanh_deriv = 1 - hiddens[1,:][np.newaxis, :] ** 2
h1_grad = np.multiply(h1_grad, tanh_deriv)

h_weight_grad += hiddens[1,:][:, np.newaxis] @ h1_grad
h_bias_grad += np.mean(h1_grad)

i_weight_grad += sequence[1].reshape(1, 1).T @ h1_grad

In [45]:
l0_grad = loss_grad[0].reshape(1,1)

o_weight_grad += hiddens[0][:, np.newaxis] @ l0_grad
o_bias_grad += np.mean(l0_grad)

h0_grad = l0_grad @o_weight.T
h0_grad += h1_grad @ h_weight.T

tanh_deriv = 1 - hiddens[0,:][np.newaxis,:] ** 2
h0_grad = np.multiply(h0_grad, tanh_deriv)

i_weight_grad += sequence[0].reshape(1, 1).T @ h0_grad

In [46]:
i_weight_grad

array([[54498.71460476,   926.96931526, 16257.75098388, 66153.99689818,
        27292.28884188]])

### Fullbackward pass in a loop


In [47]:
next_hidden = None

o_weight_grad, o_bias_grad, h_weight_grad, h_bias_grad, i_weight_grad = [0] * 5

for i in range(2, -1, -1):
    l_grad = loss_grad[i].reshape(1,1)

    o_weight_grad += hiddens[i][:, np.newaxis] @ l_grad
    o_bias_grad += np.mean(l_grad)

    o_grad = l_grad @ o_weight.T


    if next_hidden is not None:
        h_grad = o_grad + next_hidden @ h_weight.T
    else:
        h_grad = o_grad

    tanh_deriv = 1 - hiddens[i, :][np.newaxis, :] ** 2
    h_grad = np.multiply(h_grad, tanh_deriv)

    next_hidden = h_grad

    if i > 0:
        h_weight_grad += hiddens[i-1,:][:, np.newaxis] @ h_grad
        h_bias_grad += np.mean(h_grad)

    i_weight_grad += sequence[i].reshape(1, 1).T @ h_grad

In [48]:
lr = 1e-6

lr = lr /3

i_weight -= i_weight_grad *lr
h_weight -= h_weight_grad *lr
h_bias -= h_bias_grad * lr
o_weight -= o_weight_grad * lr
o_bias -= o_bias_grad * lr

## Complete Implementation

In [50]:
from sklearn.preprocessing import StandardScaler
import math 

PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

np.random.seed(0)
split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[d[PREDICTORS].to_numpy(), d[[TARGET]].to_numpy()] for d in split_data]

/home/mavis021/Documents/fuse/env/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [63]:
def init_params(layer_conf):
    layers = []
    for i in range(1, len(layer_conf)):
        np.random.seed(0)
        k = 1/math.sqrt(layer_conf[i]["hidden"])
        i_weight = np.random.rand(layer_conf[i-1]["units"], layer_conf[i]["hidden"]) * 2 * k - k

        h_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["hidden"]) * 2 * k - k
        h_bias = np.random.rand(1, layer_conf[i]["hidden"]) * 2 * k - k

        o_weight = np.random.rand(layer_conf[i]["hidden"], layer_conf[i]["output"]) * 2 * k - k
        o_bias = np.random.rand(1, layer_conf[i]["output"]) * 2 * k - k

        layers.append(
            [i_weight, h_weight, h_bias, o_weight, o_bias]
        )
    return layers

In [67]:
def forward(x, layers):
    hiddens = []
    outputs = []
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = np.zeros((x.shape[0], i_weight.shape[1]))
        output = np.zeros((x.shape[0], o_weight.shape[1]))
        for j in range(x.shape[0]):
            input_x = x[j,:][np.newaxis,:] @ i_weight
            hidden_x = input_x + hidden[max(j-1,0),:][np.newaxis,:] @ h_weight + h_bias
            # Activation.  tanh avoids outputs getting larger and larger.
            hidden_x = np.tanh(hidden_x)
            # Store hidden for use in backprop
            hidden[j,:] = hidden_x

            # Output layer
            output_x = hidden_x @ o_weight + o_bias
            output[j,:] = output_x
        hiddens.append(hidden)
        outputs.append(output)
    return hiddens, outputs[-1]

In [70]:
def backward(layers, x, lr, grad, hiddens):
    for i in range(len(layers)):
        i_weight, h_weight, h_bias, o_weight, o_bias = layers[i]
        hidden = hiddens[i]
        next_h_grad = None
        i_weight_grad, h_weight_grad, h_bias_grad, o_weight_grad, o_bias_grad = [0] * 5

        for j in range(x.shape[0] - 1, -1, -1):
            # Add newaxis in the first dimension
            out_grad = grad[j,:][np.newaxis, :]

            # Output updates
            # np.newaxis creates a size 1 axis, in this case transposing matrix
            o_weight_grad += hidden[j,:][:, np.newaxis] @ out_grad
            o_bias_grad += out_grad

            # Propagate gradient to hidden unit
            h_grad = out_grad @ o_weight.T

            if j < x.shape[0] - 1:
                # Then we multiply the gradient by the hidden weights to pull gradient from next hidden state to current hidden state
                hh_grad = next_h_grad @ h_weight.T
                # Add the gradients together to combine output contribution and hidden contribution
                h_grad += hh_grad

            # Pull the gradient across the current hidden nonlinearity
            # derivative of tanh is 1 - tanh(x) ** 2
            # So we take the output of tanh (next hidden state), and plug in
            tanh_deriv = 1 - hidden[j][np.newaxis,:] ** 2

            # next_h_grad @ np.diag(tanh_deriv_next) multiplies each element of next_h_grad by the deriv
            # Effect is to pull value across nonlinearity
            h_grad = np.multiply(h_grad, tanh_deriv)

            # Store to compute h grad for previous sequence position
            next_h_grad = h_grad.copy()

            # If we're not at the very beginning
            if j > 0:
                # Multiply input from previous layer by post-nonlinearity grad at current layer
                h_weight_grad += hidden[j-1][:, np.newaxis] @ h_grad
                h_bias_grad += h_grad

            i_weight_grad += x[j,:][:,np.newaxis] @ h_grad

        # Normalize lr by number of sequence elements
        lr = lr / x.shape[0]
        i_weight -= i_weight_grad *lr
        h_weight -= h_weight_grad *lr
        h_bias -= h_bias_grad * lr
        o_weight -= o_weight_grad * lr
        o_bias -= o_bias_grad * lr
        layers[i] = [i_weight, h_weight, h_bias, o_weight, o_bias]

    return layers

In [71]:
epochs = 200
lr = 1e-5

layer_conf = [
    {"type": "input", "units":3},
    {"type": "rnn", "hidden":4, "output":1}
]

layers = init_params(layer_conf)

for epoch in range(epochs):
    sequence_len = 7
    epoch_loss = 0

    for j in range(train_x.shape[0] - sequence_len):
        seq_x = train_x[j: (j+sequence_len), ]
        seq_y = train_y[j:(j+sequence_len), ]
        hiddens, outputs = forward(seq_x, layers)
        grad = mse_grad(seq_y, outputs)

        params = backward(layers, seq_x, lr, grad, hiddens)
        epoch_loss += mse(seq_y, outputs)

        if epoch % 50 == 0:
            sequence_len = 7
            valid_loss = 0
            for j in range(valid_x.shape[0] - sequence_len):
                seq_x = valid_x[j:(j+sequence_len), ]
                seq_y = valid_y[j: (j+sequence_len), ]
                _, outputs = forward(seq_x, layers)
                valid_loss += mse(seq_y, outputs)

            print(f"Epoch: {epoch} train loss {epoch_loss / len(train_x)} valid loss {valid_loss / len(valid_x)}")

Epoch: 0 train loss 0.28957514832895254 valid loss 4538.779041895566
Epoch: 0 train loss 0.5824343291195702 valid loss 4538.711518986511
Epoch: 0 train loss 0.8837555111080636 valid loss 4538.644487830829
Epoch: 0 train loss 1.1913319732908165 valid loss 4538.575138888427
Epoch: 0 train loss 1.5091334373784322 valid loss 4538.503977953283
Epoch: 0 train loss 1.8412618134228096 valid loss 4538.4272026141225
Epoch: 0 train loss 2.183090323327244 valid loss 4538.344251557859
Epoch: 0 train loss 2.5257560886589294 valid loss 4538.2621454042965
Epoch: 0 train loss 2.8804431446260046 valid loss 4538.177828487702
Epoch: 0 train loss 3.2385257654448485 valid loss 4538.090031107397
Epoch: 0 train loss 3.597949332374011 valid loss 4538.002300954809
Epoch: 0 train loss 3.9605585420839864 valid loss 4537.909636550961
Epoch: 0 train loss 4.32520508825161 valid loss 4537.815809298901
Epoch: 0 train loss 4.6960281171878755 valid loss 4537.723145817425
Epoch: 0 train loss 5.077805473671828 valid loss 